In [ ]:
print("=== Checking GPU Support ===")

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")

    # Clear GPU memory
    torch.cuda.empty_cache()
    print(" GPU cache cleared")

else:
    print("CUDA not available!")
print("\n" + "="*60)

In [ ]:
#IMPORTS
import torch
import torchvision
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from tqdm import tqdm
import time

print(f"PyTorch verzija: {torch.__version__}")
print(f"Torchvision verzija: {torchvision.__version__}")

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(15),                  
    transforms.RandomHorizontalFlip(p=0.5),          
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),  
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),    
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.1))  
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])



In [ ]:
# RESNET50 S TRANSFER LEARNING 
print("=" * 60)
print("ResNet50 Model for Alzheimer's Classification")
print("=" * 60)

from torchvision import models
import torch.nn as nn

class AlzheimersResNet50(nn.Module):
    def __init__(self, num_classes=4, pretrained=True):
        super(AlzheimersResNet50, self).__init__()
        

        self.resnet = models.resnet50(pretrained=pretrained)
        

        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            nn.Dropout(0.7),  
            nn.Linear(num_features, 256),  
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.7),
            nn.Linear(256, num_classes)
        )
        
        
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        for param in self.resnet.fc.parameters():
            param.requires_grad = True
    
    def forward(self, x):
        return self.resnet(x)
    
    def unfreeze_all(self):
        for param in self.resnet.parameters():
            param.requires_grad = True
    
    def unfreeze_last_layers(self, num_layers=2):
        for param in self.resnet.fc.parameters():
            param.requires_grad = True
            
        if num_layers >= 1:
            for param in self.resnet.layer4.parameters():
                param.requires_grad = True
                
        if num_layers >= 2:
            for param in self.resnet.layer3.parameters():
                param.requires_grad = True
                
        if num_layers >= 3:
            for param in self.resnet.layer2.parameters():
                param.requires_grad = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

print("Loading pre-trained ResNet50...")
model = AlzheimersResNet50(num_classes=4, pretrained=True)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("ResNet50 model created!")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Frozen parameters: {total_params - trainable_params:,}")
print("=" * 60)

In [ ]:
from torch.utils.data import Subset


full_dataset = datasets.ImageFolder(root='Data')


dataset_size = len(full_dataset)
indices = list(range(dataset_size))
np.random.shuffle(indices)

train_split = int(0.70 * dataset_size)
val_split = int(0.85 * dataset_size)

train_indices = indices[:train_split]
val_indices = indices[train_split:val_split]
test_indices = indices[val_split:]


train_dataset = Subset(datasets.ImageFolder(root='Data', transform=train_transform), train_indices)
val_dataset = Subset(datasets.ImageFolder(root='Data', transform=val_transform), val_indices)
test_dataset = Subset(datasets.ImageFolder(root='Data', transform=val_transform), test_indices)



batch_size = 32 

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=False)

print(f"Dataset is splitted:")
print(f"   Training set: {len(train_dataset)} images ({len(train_dataset)/len(full_dataset)*100:.1f}%)")
print(f"   Validation set: {len(val_dataset)} images ({len(val_dataset)/len(full_dataset)*100:.1f}%)")
print(f"   Test set: {len(test_dataset)} images ({len(test_dataset)/len(full_dataset)*100:.1f}%)")
print(f"    Batch size: {batch_size}")

print(f"\n Classes in dataset: {full_dataset.classes}")
print(f" Class mapping: {full_dataset.class_to_idx}")

In [ ]:
print("SETTING UP TRAINING FOR RESNET50")
print("=" * 50)

criterion = nn.CrossEntropyLoss(label_smoothing=0.2)  


optimizer = optim.AdamW(model.parameters(), lr=0.00001, weight_decay=0.05)

scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.3, patience=3)

train_losses = []
train_accuracies = []
val_accuracies = []

print(f"Loss function: CrossEntropyLoss")
print(f"Optimizer: Adam (lr=0.0001)")
print(f"Scheduler: ReduceLROnPlateau ")

print("\nTesting ResNet50 with sample batch")
try:
    data_iter = iter(train_loader)
    test_images, test_labels = next(data_iter)
    test_images = test_images.to(device)
    test_labels = test_labels.to(device)

    print(f"Loaded batch of {test_images.shape[0]} images")
    
    model.eval()
    with torch.no_grad():
        test_output = model(test_images)

    print(f"ResNet50 test passed successfully!")
    print(f"Input shape: {test_images.shape}")
    print(f"Output shape: {test_output.shape}")
    print(f"Number of classes in output: {test_output.shape[1]}")
    print(f"Sample labels: {test_labels[:5].cpu().numpy()}")
    
    model.train()

    print("ResNet50 is ready for training")

except Exception as e:
    print(f"Error in ResNet50 model: {e}")
    print(f"Error type: {type(e).__name__}")
    import traceback
    traceback.print_exc()

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    train_bar = tqdm(train_loader, desc='Training')
    
    for batch_idx, (images, labels) in enumerate(train_bar):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
        current_acc = 100. * correct_predictions / total_samples
        train_bar.set_postfix({
            'Loss': f'{running_loss/(batch_idx+1):.4f}',
            'Acc': f'{current_acc:.2f}%'
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct_predictions / total_samples
    
    return epoch_loss, epoch_acc

def validate_epoch(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc='Validation')
        
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            
            current_acc = 100. * correct_predictions / total_samples
            val_bar.set_postfix({
                'Loss': f'{running_loss/(len(val_bar.iterable)):.4f}',
                'Acc': f'{current_acc:.2f}%'
            })
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct_predictions / total_samples
    
    return epoch_loss, epoch_acc


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, 
                num_epochs=20, device='cpu'):
    
    print(f"Starting training on {device}")
    print(f"Number of epochs: {num_epochs}")
    print(f"Training batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")
    print("="*60)
    
    
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []
    
    best_val_acc = 0.0
    start_time = time.time()
    
    for epoch in range(num_epochs):
        epoch_start = time.time()
        print(f"\nEPOHA {epoch+1}/{num_epochs}")
        print("-" * 40)
        
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
        
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        scheduler.step(val_acc)
        
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_alzheimers_model.pth')
            print(f"New best model saved. Accuracy: {val_acc:.2f}%")

        epoch_time = time.time() - epoch_start
        print(f"\nEpoch {epoch+1} results:")
        print(f"    Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"    Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        print(f"    Epoch time: {epoch_time:.1f}s")
        print(f"    Best val accuracy: {best_val_acc:.2f}%")

        if epoch > 5 and val_acc < best_val_acc - 10:
            print(f"Early stopping - no improvement")
            break
    
    total_time = time.time() - start_time
    print("="*60)
    print(f"Training completed!")
    print(f"Total time: {total_time/60:.1f} minutes")
    print(f"Best validation accuracy: {best_val_acc:.2f}%")

    return train_losses, train_accuracies, val_losses, val_accuracies

print("Main training function defined.")

In [ ]:
print("Starting training")
print("=" * 50)

num_epochs = 15
print(f"Number of epochs: {num_epochs}")
print(f"Batch size: {batch_size}")
print(f"Device: {device}")

print("\nStarting training...")
print("="*60)

try:

    train_losses, train_accs, val_losses, val_accs = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=num_epochs,
        device=device
    )

    print("Training completed.")

except RuntimeError as e:
    if "out of memory" in str(e):
        print("GPU has not enough memory!")
        print("Switching to CPU:")

        # Move model to CPU
        device = torch.device('cpu')
        model = model.to(device)
        print(f"Model moved to CPU")
        print("Training will be slower but more stable")

        # Start training on CPU
        train_losses, train_accs, val_losses, val_accs = train_model(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=scheduler,
            num_epochs=num_epochs,
            device=device
        )
    else:
        print(f"Unexpected error: {e}")
        raise e

In [ ]:
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []

    print("Evaluating on test set:")

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='Testing'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_predictions)
    class_names = ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - Accuracy: {accuracy:.2f}%')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    return accuracy

# Ucitaj najbolji model i evaluiraj
model.load_state_dict(torch.load('best_alzheimers_model.pth'))
test_accuracy = evaluate_model(model, test_loader, device)

print(f"Final test accuracy: {test_accuracy:.2f}%")

In [ ]:
def predict_image(model, image_path, device):
    
    from PIL import Image
    
    image = Image.open(image_path).convert('RGB')
    
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        probabilities = F.softmax(outputs, dim=1)
        confidence, predicted = torch.max(probabilities, 1)
    
    class_names = ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
    predicted_class = class_names[predicted.item()]
    confidence_score = confidence.item() * 100
    
    all_probs = probabilities[0].cpu().numpy()
    class_probabilities = {class_names[i]: all_probs[i] * 100 for i in range(len(class_names))}
    
    return predicted_class, confidence_score, class_probabilities


torch.save(model.state_dict(), 'final_alzheimers_model.pth')
print("Model saved as 'final_alzheimers_model.pth'")



In [ ]:
print("=" * 60)
print("DETAILED DATASET ANALYSIS")
print("=" * 60)

import os
import hashlib
from PIL import Image
from collections import defaultdict, Counter
import numpy as np

def get_image_hash(image_path):
    try:
        with Image.open(image_path) as img:
            img = img.convert('L').resize((64, 64))
            img_array = np.array(img)
            return hashlib.md5(img_array.tobytes()).hexdigest()
    except:
        return None

def analyze_dataset(data_path):
    
    class_counts = {}
    image_hashes = defaultdict(list)
    total_images = 0
    
    
    for class_name in os.listdir(data_path):
        class_path = os.path.join(data_path, class_name)
        if os.path.isdir(class_path):
            image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            class_counts[class_name] = len(image_files)
            total_images += len(image_files)

            print(f"Analyzing {class_name}: {len(image_files)} images...")

            for img_file in image_files[:500]:  
                img_path = os.path.join(class_path, img_file)
                img_hash = get_image_hash(img_path)
                if img_hash:
                    image_hashes[img_hash].append((class_name, img_file))
    

    print(f"\n DATASET STATISTICS:")
    print(f"Total images: {total_images}")
    print(f"Number of classes: {len(class_counts)}")

    print(f"\n CLASS DISTRIBUTION:")
    for class_name, count in sorted(class_counts.items()):
        percentage = (count / total_images) * 100
        print(f"   {class_name}: {count:,} images ({percentage:.1f}%)")

    # Check for balance
    counts = list(class_counts.values())
    min_count, max_count = min(counts), max(counts)
    imbalance_ratio = max_count / min_count

    print(f"\n BALANCE:")
    print(f"   Min images per class: {min_count:,}")
    print(f"   Max images per class: {max_count:,}")
    print(f"   Imbalance ratio: {imbalance_ratio:.2f}")
    
    if imbalance_ratio > 10:
        print("   High imbalance - class weighting or resampling")
    elif imbalance_ratio > 3:
        print("   Moderate imbalance - class weighting")
    else:
        print("   Dataset is fairly balanced")

    duplicates = {h: files for h, files in image_hashes.items() if len(files) > 1}

    print(f"\n DUPLICATES:")
    if duplicates:
        print(f"   Found {len(duplicates)} groups of duplicates")
        total_duplicate_images = sum(len(files) for files in duplicates.values())
        print(f"   Total duplicate images: {total_duplicate_images}")
        print("   WARNING: Duplicates may cause data leakage!")

        for i, (hash_val, files) in enumerate(list(duplicates.items())[:3]):
            print(f"   Example {i+1}: {len(files)} identical images")
            for class_name, filename in files[:3]:
                print(f"      - {class_name}/{filename}")
    else:
        print("   No duplicates found")

    return class_counts, duplicates


try:
    class_counts, duplicates = analyze_dataset('Data')
except Exception as e:
    print(f"Error analyzing dataset: {e}")
    print("Continuing without detailed analysis...")

In [ ]:
print("=" * 60)
print("CREATING OPTIMALLY REGULARIZED RESNET50 MODEL")
print("=" * 60)

from torchvision import models
import torch.nn as nn
import torch.nn.functional as F

class OptimalAlzheimersResNet50(nn.Module):
    def __init__(self, num_classes=4, pretrained=True, dropout_rate=0.5):
        super(OptimalAlzheimersResNet50, self).__init__()
        

        self.resnet = models.resnet50(pretrained=pretrained)
        

        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()
        
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),  # 0.5
            
            # Prvi FC sloj
            nn.Linear(num_features, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate * 0.8),  # 0.4
            
            # Drugi FC sloj
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate * 0.6),  # 0.3
            
            # Treći FC sloj
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate * 0.4),  # 0.2
            
            # Finalni sloj
            nn.Linear(256, num_classes)
        )
        
        self._initialize_weights()
        
        self._freeze_backbone(freeze_percentage=0.7)
    
    def _initialize_weights(self):
        for m in self.classifier.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def _freeze_backbone(self, freeze_percentage=0.7):
        total_layers = len(list(self.resnet.parameters()))
        layers_to_freeze = int(total_layers * freeze_percentage)
        
        for i, param in enumerate(self.resnet.parameters()):
            if i < layers_to_freeze:
                param.requires_grad = False
            else:
                param.requires_grad = True
    
    def forward(self, x):
        # Feature extraction
        features = self.resnet(x)  # [batch_size, 2048]
        
        features = F.normalize(features, p=2, dim=1)
        
        output = self.classifier(features)
        return output
    
    def unfreeze_more_layers(self, unfreeze_percentage=0.3):
        total_layers = len(list(self.resnet.parameters()))
        layers_to_unfreeze = int(total_layers * unfreeze_percentage)
        
        resnet_params = list(self.resnet.parameters())
        for i in range(max(0, total_layers - layers_to_unfreeze), total_layers):
            resnet_params[i].requires_grad = True
    
    def get_trainable_params(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

print("Creating optimally regularized ResNet50")
optimal_model = OptimalAlzheimersResNet50(num_classes=4, dropout_rate=0.5)

total_params = sum(p.numel() for p in optimal_model.parameters())
trainable_params = optimal_model.get_trainable_params()

print(f" Total parameters: {total_params:,}")
print(f" Trainable parameters: {trainable_params:,}")
print(f" Frozen: {total_params - trainable_params:,} ({((total_params - trainable_params)/total_params)*100:.1f}%)")
print(f" Optimal regularization: Dropout 0.5, 4 layers, BatchNorm, L2 Norm")
print("=" * 60)

In [ ]:
print("=" * 60)
print("CREATING BALANCED TRAIN/VAL/TEST SPLIT")
print("=" * 60)

from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import numpy as np

def create_stratified_split(dataset, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=42):
  
    targets = [dataset[i][1] for i in range(len(dataset))]
    indices = list(range(len(dataset)))
    
    # Stratified split: train + (val+test)
    train_indices, temp_indices, train_targets, temp_targets = train_test_split(
        indices, targets, 
        test_size=(val_ratio + test_ratio),
        stratify=targets,
        random_state=random_state
    )
    
    # Split temp na val i test
    val_test_ratio = val_ratio / (val_ratio + test_ratio)
    val_indices, test_indices = train_test_split(
        temp_indices,
        test_size=(1 - val_test_ratio),
        stratify=temp_targets,
        random_state=random_state
    )
    
    return train_indices, val_indices, test_indices



try:
    original_dataset = datasets.ImageFolder(root='Data')
    

    train_idx, val_idx, test_idx = create_stratified_split(
        original_dataset,
        train_ratio=0.70,
        val_ratio=0.15,
        test_ratio=0.15,
        random_state=42
    )

    print(f" Stratified split created.")
    print(f" Training set: {len(train_idx):,} images ({len(train_idx)/len(original_dataset)*100:.1f}%)")
    print(f" Validation set: {len(val_idx):,} images ({len(val_idx)/len(original_dataset)*100:.1f}%)")
    print(f" Test set: {len(test_idx):,} images ({len(test_idx)/len(original_dataset)*100:.1f}%)")

    def check_class_distribution(dataset, indices, split_name):
        labels = [dataset[i][1] for i in indices]
        class_counts = Counter(labels)
        total = len(indices)

        print(f"\n {split_name} distribution:")
        for class_idx, class_name in enumerate(dataset.classes):
            count = class_counts[class_idx]
            percentage = (count / total) * 100
            print(f"   {class_name}: {count:,} ({percentage:.1f}%)")
    
    check_class_distribution(original_dataset, train_idx, "TRAIN")
    check_class_distribution(original_dataset, val_idx, "VALIDATION") 
    check_class_distribution(original_dataset, test_idx, "TEST")
    

    print(f"\n Creating augmented transformations.")

    robust_train_transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomRotation(15),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2, hue=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

        transforms.RandomErasing(p=0.2, scale=(0.02, 0.15), ratio=(0.3, 3.3))
    ])
    

    robust_val_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    

    balanced_train_dataset = Subset(
        datasets.ImageFolder(root='Data', transform=robust_train_transform),
        train_idx
    )
    
    balanced_val_dataset = Subset(
        datasets.ImageFolder(root='Data', transform=robust_val_transform),
        val_idx
    )
    
    balanced_test_dataset = Subset(
        datasets.ImageFolder(root='Data', transform=robust_val_transform),
        test_idx
    )
    
    print(f" Enhanced datasets created with stronger augmentations.")

except Exception as e:
    print(f"Error: {e}")
    print("Continuing with basic split...")

    # Fallback na originalni pristup
    balanced_train_dataset = train_dataset
    balanced_val_dataset = val_dataset  
    balanced_test_dataset = test_dataset
    robust_train_transform = train_transform
    robust_val_transform = val_transform

print("=" * 60)

In [ ]:
print("=" * 60)
print("SETUP FOR OPTIMAL TRAINING")
print("=" * 60)

from torch.utils.data import WeightedRandomSampler
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, OneCycleLR

def create_balanced_dataloader(dataset, batch_size=12, num_workers=0):  # Smanjeno za stabilnost
    

    targets = [dataset[i][1] for i in range(len(dataset))]
    class_counts = Counter(targets)
    total_samples = len(targets)
    
    class_weights = {cls: total_samples / count for cls, count in class_counts.items()}
    

    sample_weights = [class_weights[target] for target in targets]
    

    sampler = WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(sample_weights),
        replacement=True
    )
    
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=sampler,
        num_workers=num_workers,
        pin_memory=torch.cuda.is_available(),
        drop_last=True  
    )
    
    return dataloader


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f" Using device: {device}")


optimal_model = optimal_model.to(device)


optimal_batch_size = 12 
print(f" Batch size: {optimal_batch_size}")

print("Creating optimal data loaders.")
try:

    optimal_train_loader = create_balanced_dataloader(
        balanced_train_dataset, 
        batch_size=optimal_batch_size,
        num_workers=0
    )
    

    optimal_val_loader = torch.utils.data.DataLoader(
        balanced_val_dataset,
        batch_size=optimal_batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available()
    )
    
    optimal_test_loader = torch.utils.data.DataLoader(
        balanced_test_dataset,
        batch_size=optimal_batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=torch.cuda.is_available()
    )
    
    print(f" Optimal data loaders created!")
    print(f"   Training batches: {len(optimal_train_loader)}")
    print(f"   Validation batches: {len(optimal_val_loader)}")
    print(f"   Test batches: {len(optimal_test_loader)}")
    
except Exception as e:
    print(f" Error with optimal loaders: {e}")
    optimal_train_loader = torch.utils.data.DataLoader(
        balanced_train_dataset, batch_size=optimal_batch_size, 
        shuffle=True, num_workers=0
    )
    optimal_val_loader = torch.utils.data.DataLoader(
        balanced_val_dataset, batch_size=optimal_batch_size, 
        shuffle=False, num_workers=0
    )
    optimal_test_loader = torch.utils.data.DataLoader(
        balanced_test_dataset, batch_size=optimal_batch_size, 
        shuffle=False, num_workers=0
    )

print("SETTING UP OPTIMAL TRAINING")
# Weighted CrossEntropy with moderate label smoothing
if 'class_counts' in locals():
    # Calculate class weights
    total = sum(class_counts.values())
    weights = torch.tensor([total/count for count in class_counts.values()], 
                          dtype=torch.float32, device=device)
    criterion = nn.CrossEntropyLoss(weight=weights, label_smoothing=0.1)  # Moderate smoothing
    print(f" Class weights: {weights.cpu().numpy().round(2)}")
else:
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# AdamW optimizer 
optimizer = optim.AdamW(
    optimal_model.parameters(),
    lr=3e-5,  # Optimal learning rate
    weight_decay=0.01,  # Moderate weight decay
    betas=(0.9, 0.999),
    eps=1e-8
)

# Optimal scheduler
total_steps = len(optimal_train_loader) * 30  # 30 epoha
scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=len(optimal_train_loader) * 6,  # Restart svake 6 epoha
    T_mult=1,
    eta_min=1e-7
)

print(f" Optimal configuration set!")
print(f" Loss: CrossEntropyLoss with label smoothing 0.1")
print(f" Optimizer: AdamW (lr=3e-5, wd=0.01)")
print(f" Scheduler: CosineAnnealingWarmRestarts")
print(f" Batch size: {optimal_batch_size}")
print("=" * 60)

In [ ]:
print("="*80)
print(" PHASE 2: PROGRESSIVE UNFREEZING FOR BETTER ACCURACY")
print("="*80)

try:
    print(f" Checking existing variables:")
    print(f" Model: {type(model).__name__}")
    print(f" Device: {device}")
    print(f" Train loader: {len(train_loader)} batches")
    print(f" Val loader: {len(val_loader)} batches")
    print(" All variables are available!")
except NameError as e:
    print(f" Missing variable: {e}")
    raise e


print("=" * 60)
print("CREATING OPTIMAL TRAINING FUNCTION")
print("=" * 60)

import time
import copy
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support, classification_report

class OptimalTrainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer, scheduler, device):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        
        # Tracking
        self.train_losses = []
        self.train_accuracies = []
        self.val_losses = []
        self.val_accuracies = []
        self.learning_rates = []
        
        # Early stopping - balansirano
        self.best_val_acc = 0.0
        self.best_model_state = None
        self.patience_counter = 0
        
    def train_epoch(self):
        self.model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        
        train_bar = tqdm(self.train_loader, desc='Training')
        
        for batch_idx, (images, labels) in enumerate(train_bar):
            images, labels = images.to(self.device), labels.to(self.device)
            
            # Forward pass
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, labels)

            l2_lambda = 0.005  # Manja nego zadnja verzija
            l2_norm = sum(p.pow(2.0).sum() for p in self.model.parameters() if p.requires_grad)
            loss = loss + l2_lambda * l2_norm
            
            # Backward pass
            loss.backward()
            
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            
            self.optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()
            
            current_acc = 100. * correct_predictions / total_samples
            current_lr = self.optimizer.param_groups[0]['lr']
            
            train_bar.set_postfix({
                'Loss': f'{running_loss/(batch_idx+1):.4f}',
                'Acc': f'{current_acc:.2f}%',
                'LR': f'{current_lr:.2e}'
            })
        
        epoch_loss = running_loss / len(self.train_loader)
        epoch_acc = 100. * correct_predictions / total_samples
        epoch_lr = self.optimizer.param_groups[0]['lr']
        
        return epoch_loss, epoch_acc, epoch_lr
    
    def validate_epoch(self):
        self.model.eval()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0
        all_predictions = []
        all_labels = []
        
        with torch.no_grad():
            val_bar = tqdm(self.val_loader, desc='Validation')
            
            for images, labels in val_bar:
                images, labels = images.to(self.device), labels.to(self.device)
                
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                
                running_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_samples += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()
                
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                
                current_acc = 100. * correct_predictions / total_samples
                val_bar.set_postfix({
                    'Loss': f'{running_loss/(len(all_predictions)//labels.size(0)):.4f}',
                    'Acc': f'{current_acc:.2f}%'
                })
        
        epoch_loss = running_loss / len(self.val_loader)
        epoch_acc = 100. * correct_predictions / total_samples
        
        precision, recall, f1, support = precision_recall_fscore_support(
            all_labels, all_predictions, average=None, zero_division=0
        )
        
        return epoch_loss, epoch_acc, precision, recall, f1
    
    def train(self, num_epochs=20, patience=8, save_path='optimal_alzheimers_model.pth'):
        print(f" Starting optimal training on {self.device}")
        print(f" Epochs: {num_epochs}, Early stopping patience: {patience}")
        print("=" * 60)
        
        start_time = time.time()
        
        for epoch in range(num_epochs):
            epoch_start = time.time()
            print(f"\n EPOHA {epoch+1}/{num_epochs}")
            print("-" * 50)
            
            # Training
            train_loss, train_acc, lr = self.train_epoch()
            
            # Validation
            val_loss, val_acc, precision, recall, f1 = self.validate_epoch()
            
            # Tracking
            self.train_losses.append(train_loss)
            self.train_accuracies.append(train_acc)
            self.val_losses.append(val_loss)
            self.val_accuracies.append(val_acc)
            self.learning_rates.append(lr)
            
            if val_acc > self.best_val_acc and (train_acc - val_acc) < 8:  # Dozvoljen gap do 8%
                self.best_val_acc = val_acc
                self.best_model_state = copy.deepcopy(self.model.state_dict())
                self.patience_counter = 0
                
                torch.save(self.best_model_state, save_path)
                improvement_msg = f" New best: Model with {val_acc:.2f}% val acc saved."
            else:
                self.patience_counter += 1
                improvement_msg = f" Best: {self.best_val_acc:.2f}% (patience: {self.patience_counter}/{patience})"
            
            epoch_time = time.time() - epoch_start
            overfitting_gap = train_acc - val_acc
            
            print(f"\n RESULTS FOR EPOCH {epoch+1}:")
            print(f"   Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
            print(f"   Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
            print(f"   Gap: {overfitting_gap:.2f}% {'(excellent)' if overfitting_gap < 5 else '(good)' if overfitting_gap < 8 else '(caution)'}")
            print(f"   Avg F1: {f1.mean():.3f}, LR: {lr:.2e}")
            print(f"   Time: {epoch_time:.1f}s")
            print(f"   {improvement_msg}")
            
            if hasattr(self.model, 'unfreeze_more_layers'):
                if epoch == 5 and val_acc > 80:
                    print(f"\n Unfreezing more layers - Phase 1")
                    self.model.unfreeze_more_layers(0.3)  # 30%
                    trainable = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
                    print(f"   Trainable parameters: {trainable:,}")

                elif epoch == 12 and val_acc > 85:
                    print(f"\n Unfreezing more layers - Phase 2")
                    self.model.unfreeze_more_layers(0.5)  # 50%
                    trainable = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
                    print(f"   Trainable parameters: {trainable:,}")

                    for param_group in self.optimizer.param_groups:
                        param_group['lr'] *= 0.5
                    print(f"   Learning rate reduced to: {self.optimizer.param_groups[0]['lr']:.2e}")

            if self.patience_counter >= patience:
                print(f"\n EARLY STOPPING after {epoch+1} epochs")
                print(f"   No improvement for {patience} epochs")
                break
                
            if overfitting_gap > 15:
                print(f"\n Overfitting detected --> reducing learning rate")
                for param_group in self.optimizer.param_groups:
                    param_group['lr'] *= 0.7
                print(f"   New LR: {self.optimizer.param_groups[0]['lr']:.2e}")

        total_time = time.time() - start_time
        
        print("\n" + "=" * 60)
        print(f" Training done:")
        print(f" Total time: {total_time/60:.1f} minutes")
        print(f" Best validation accuracy: {self.best_val_acc:.2f}%")
        print(f" Model saved to: {save_path}")
        
        if self.best_val_acc >= 88:
            print(f" Target achieved: {self.best_val_acc:.2f}%")
        elif self.best_val_acc >= 85:
            print(f" Close to target")
        else:
            print(f" Fine-tuning needed")


        self.model.load_state_dict(self.best_model_state)
        
        return self.train_losses, self.train_accuracies, self.val_losses, self.val_accuracies
    
    def plot_training_history(self):
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        epochs = range(1, len(self.train_losses) + 1)
        
        ax1.plot(epochs, self.train_losses, 'bo-', label='Train Loss')
        ax1.plot(epochs, self.val_losses, 'ro-', label='Val Loss')
        ax1.set_title('Training and Validation Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        
        # Accuracy
        ax2.plot(epochs, self.train_accuracies, 'bo-', label='Train Acc')
        ax2.plot(epochs, self.val_accuracies, 'ro-', label='Val Acc')
        ax2.axhline(y=88, color='g', linestyle='--', alpha=0.7)
        ax2.axhline(y=90, color='orange', linestyle='--', alpha=0.7)
        ax2.set_title('Training and Validation Accuracy')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('Accuracy (%)')
        ax2.legend()
        ax2.grid(True)
        
        # Learning Rate
        ax3.plot(epochs, self.learning_rates, 'go-', label='Learning Rate')
        ax3.set_title('Learning Rate Schedule')
        ax3.set_xlabel('Epoch')
        ax3.set_ylabel('Learning Rate')
        ax3.set_yscale('log')
        ax3.legend()
        ax3.grid(True)
        
        # Overfitting check
        train_val_diff = [t - v for t, v in zip(self.train_accuracies, self.val_accuracies)]
        ax4.plot(epochs, train_val_diff, 'mo-', label='Train - Val Acc')
        ax4.axhline(y=0, color='k', linestyle='--', alpha=0.5)
        ax4.axhline(y=5, color='g', linestyle='--', alpha=0.5)
        ax4.axhline(y=8, color='orange', linestyle='--', alpha=0.5)
        ax4.set_title('Overfitting Monitor')
        ax4.set_xlabel('Epoch')  
        ax4.set_ylabel('Accuracy Difference (%)')
        ax4.legend()
        ax4.grid(True)
        
        plt.tight_layout()
        plt.show()


progressive_optimizer = optim.AdamW(model.parameters(), lr=0.00005, weight_decay=0.01)
progressive_scheduler = optim.lr_scheduler.CosineAnnealingLR(progressive_optimizer, T_max=20)


progressive_trainer = OptimalTrainer(
    model=model,  # Koristi postojeći model
    train_loader=train_loader,  # Koristi postojeći train_loader
    val_loader=val_loader,  # Koristi postojeći val_loader
    criterion=criterion,  # Koristi postojeći criterion
    optimizer=progressive_optimizer,
    scheduler=progressive_scheduler,
    device=device
)

print(" Progressive Trainer done.")
print("=" * 60)

In [ ]:
print(" STARTING OPTIMAL TRAINING")
print("=" * 60)


# Optimalni parametri
optimal_epochs = 30
optimal_patience = 10

print(f" OPTIMAL PARAMETERS:")
print(f"   Epochs: {optimal_epochs}")
print(f"   Early stopping patience: {optimal_patience}")
print(f"   Batch size: {batch_size}")
print(f"   Learning rate: 3e-5")
print(f"   Weight decay: 0.01")
print(f"   Dropout: 0.5")
print(f"   Gradient clipping: 1.0")


# Počisti memoriju od starih modela
if 'model' in globals():
    del model
if 'ultra_model' in globals():
    del ultra_model
torch.cuda.empty_cache()

response = input("\\n Starting optimal training (y/n): ")

if response.lower() in ['y', 'yes', 'da', 'd']:
    print(f"\\n Optimal training starting:")
    print("=" * 60)
    
    try:
        # Pokreni optimalno treniranje
        optimal_train_hist, optimal_train_acc_hist, optimal_val_hist, optimal_val_acc_hist = optimal_trainer.train(
            num_epochs=optimal_epochs,
            patience=optimal_patience,
            save_path='optimal_alzheimers_model.pth'
        )

        print("\\n Optimal training finished.")

        # Analyze results
        final_val_acc = optimal_val_acc_hist[-1] if optimal_val_acc_hist else 0
        best_val_acc = max(optimal_val_acc_hist) if optimal_val_acc_hist else 0
        final_train_acc = optimal_train_acc_hist[-1] if optimal_train_acc_hist else 0
        overfitting_gap = final_train_acc - final_val_acc

        print(f"\\n FINAL RESULTS:")
        print(f"   Best Val Accuracy: {best_val_acc:.2f}%")
        print(f"   Final Val Accuracy: {final_val_acc:.2f}%")
        print(f"   Overfitting gap: {overfitting_gap:.2f}%")
        
        

        print("\\n Saving and plotting training history...")
        optimal_trainer.plot_training_history()
        

        torch.save(optimal_model.state_dict(), 'final_optimal_alzheimers_model.pth')
        print(f"\\n Final model saved as 'final_optimal_alzheimers_model.pth'")

        # Test on test set
        print("\\n Testing on test set...")
        optimal_model.eval()
        test_correct = 0
        test_total = 0
        all_test_preds = []
        all_test_labels = []
        
        with torch.no_grad():
            for images, labels in optimal_test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = optimal_model(images)
                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
                all_test_preds.extend(predicted.cpu().numpy())
                all_test_labels.extend(labels.cpu().numpy())
        
        test_accuracy = 100 * test_correct / test_total
        print(f"FINAL TEST ACCURACY: {test_accuracy:.2f}%")
        
        # Per-class test rezultati
        from sklearn.metrics import classification_report
        class_names = ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
        print("\\nPer-class test results:")
        print(classification_report(all_test_labels, all_test_preds, target_names=class_names))

        # Save per-class results
        with open("per_class_results.txt", "w") as f:
            f.write(classification_report(all_test_labels, all_test_preds, target_names=class_names))

    except KeyboardInterrupt:
        print("\\n Training interrupted by user")
        print("Model has been saved so far...")
        
    except Exception as e:
        print(f"\\n Error during training: {e}")
        print("Checking if all parameters are set correctly...")
        
        # Debugging informacije
        print(f"\\n DEBUGGING INFO:")
        print(f"   Device: {device}")
        print(f"   Model parameters: {optimal_model.get_trainable_params():,}")
        print(f"   Train loader batches: {len(optimal_train_loader)}")
        print(f"   Val loader batches: {len(optimal_val_loader)}")
        

        if "cuda" in str(e).lower() or "memory" in str(e).lower():
            print("\\n Switching to CPU due to GPU error...")
            device = torch.device('cpu')
            optimal_model = optimal_model.to(device)
            
            # Update trainer device
            optimal_trainer.device = device
            optimal_trainer.model = optimal_model

            print("Trying again with CPU")
            optimal_train_hist, optimal_train_acc_hist, optimal_val_hist, optimal_val_acc_hist = optimal_trainer.train(
                num_epochs=optimal_epochs,
                patience=optimal_patience,
                save_path='optimal_alzheimers_model_cpu.pth'
            )
            
else:
    print("\\n Training postponed.")
    print("\\n Currently have:")
    print("   - Optimal regularized model (dropout 0.5, 4 layers)")
    print("   - Optimal learning rate (3e-5)")
    print("   - Moderate weight decay (0.01)")
    print("   - Optimal batch size (12) - reduced for stability")
    print("   - 70% layers frozen")
    print("   - Gradual fine-tuning (epoch 8 and 16)")
    print("\\n" + "=" * 60)

In [ ]:
# OPTIMAL MODEL USING SGD OPTIMIZER 

print("=" * 60)
print("CREATING OPTIMAL ALZHEIMERS MODEL USING SGD OPTIMIZER")
print("=" * 60)

# SGD Model s optimalnim parametrima
sgd_model = AlzheimersResNet50(num_classes=4, dropout_rate=0.5).to(device)
print(f"Model created on: {device}")

# SGD Optimizer 
sgd_optimizer = torch.optim.SGD(
    sgd_model.parameters(),
    lr=0.01,              # Veći learning rate za SGD
    momentum=0.9,         # Ključno za SGD performanse
    weight_decay=1e-4,    # L2 regularization
    nesterov=True         # Nesterov momentum za brži i stabilniji trening
)

# Isti scheduler kao i za AdamW
sgd_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    sgd_optimizer, T_0=10, T_mult=2, eta_min=1e-6
)



In [ ]:

print("PHASE 2: PROGRESSIVE UNFREEZING")
print("=" * 70)


# Step 1: Unfreeze layer4 (last ResNet block)
print("\n STEP 1: Unfreezing layer4 (last ResNet block)")
sgd_model.unfreeze_more_layers(0.3)  # Unfreeze 30% of layers

# Count parameters after unfreezing
total_params = sum(p.numel() for p in sgd_model.parameters())
trainable_params = sum(p.numel() for p in sgd_model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,} (before: ~6,000)")
print(f"Trainable ratio: {trainable_params/total_params*100:.1f}%")

# Reduce learning rate for fine-tuning
print(f"\n Adjusting optimizer for fine-tuning:")
print(f"Learning rate: 0.00001 → 0.000005 (lowered for stability)")

# New optimizer with lower LR
optimizer = optim.AdamW(sgd_model.parameters(), lr=0.000005, weight_decay=0.05)

# Reduce number of epochs - fine-tuning is faster
fine_tuning_epochs = 10

print(f"Epochs for fine-tuning: {fine_tuning_epochs}")
print(f"\nStarting fine-tuning training.")

In [ ]:
print("=" * 70)
print("STARTING FINE-TUNING TRAINING WITH PROGRESSIVE UNFREEZING")
print("=" * 70)

import time
from sklearn.metrics import classification_report

# Scheduler za fine-tuning
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.7, patience=3
)

# Training tracking
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
best_val_acc = 0.0
best_model_state = None

print(f"Model: {sgd_model.__class__.__name__}")
print(f"Optimizer: AdamW (LR: {optimizer.param_groups[0]['lr']:.2e})")
print(f"Trainable params: {trainable_params:,} ({trainable_params/total_params*100:.1f}%)")
print("=" * 50)

start_time = time.time()

# FINE-TUNING TRAINING LOOP
for epoch in range(fine_tuning_epochs):
    epoch_start = time.time()
    print(f"\nEPOCH {epoch+1}/{fine_tuning_epochs}")
    print("-" * 40)
    
    # TRAINING PHASE
    sgd_model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    train_bar = tqdm(train_loader, desc='Training')
    
    for batch_idx, (images, labels) in enumerate(train_bar):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = sgd_model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping za stabilnost
        torch.nn.utils.clip_grad_norm_(sgd_model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels).sum().item()
        
        # Progress update
        current_acc = 100. * correct_predictions / total_samples
        train_bar.set_postfix({
            'Loss': f'{running_loss/(batch_idx+1):.4f}',
            'Acc': f'{current_acc:.2f}%'
        })
    
    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct_predictions / total_samples
    
    # VALIDATION PHASE
    sgd_model.eval()
    val_running_loss = 0.0
    val_correct = 0
    val_total = 0
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        val_bar = tqdm(val_loader, desc='Validation')
        
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            
            outputs = sgd_model(images)
            loss = criterion(outputs, labels)
            
            val_running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            current_acc = 100. * val_correct / val_total
            val_bar.set_postfix({'Acc': f'{current_acc:.2f}%'})
    
    val_loss = val_running_loss / len(val_loader)
    val_acc = 100. * val_correct / val_total
    
    # Save tracking
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # Check for best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = copy.deepcopy(sgd_model.state_dict())
        improvement_msg = "Best new record"
    else:
        improvement_msg = f"Best: {best_val_acc:.2f}%"
    
    # Learning rate scheduling
    scheduler.step(val_acc)
    current_lr = optimizer.param_groups[0]['lr']
    
    # Results summary
    overfitting_gap = train_acc - val_acc
    epoch_time = time.time() - epoch_start

    print(f"\nEpoch results {epoch+1}:")
    print(f"  Train: Loss={train_loss:.4f}, Acc={train_acc:.2f}%")
    print(f"  Val:   Loss={val_loss:.4f}, Acc={val_acc:.2f}%")
    print(f"  Gap: {overfitting_gap:.2f}% {'(excellent)' if overfitting_gap < 5 else '(good)' if overfitting_gap < 8 else '(caution)'}")
    print(f"  LR: {current_lr:.2e}, Time: {epoch_time:.1f}s")
    print(f"  {improvement_msg}")
    
    # Progressive unfreezing na polovini treninga
    if epoch == 5 and val_acc > 82:
        print(f"\nUnfreezing additional layers - Phase 2!")
        sgd_model.unfreeze_more_layers(0.5)  # 50% slojeva
        new_trainable = sum(p.numel() for p in sgd_model.parameters() if p.requires_grad)
        print(f"  New trainable parameters: {new_trainable:,}")

        # Reduce learning rate for fine-tuning
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.8
        print(f"  Learning rate reduced to: {optimizer.param_groups[0]['lr']:.2e}")

total_time = time.time() - start_time

print("\n" + "=" * 70)
print("FINE-TUNING COMPLETED!")
print("=" * 70)
print(f"Total time: {total_time/60:.1f} minutes")
print(f"Best validation accuracy: {best_val_acc:.2f}%")


# Load best model
if best_model_state:
    sgd_model.load_state_dict(best_model_state)
    print("Best model loaded!")

print(f"\nFINAL RESULTS:")
print(f"  Starting accuracy: 78.33%")
print(f"  Final accuracy: {best_val_acc:.2f}%")
print(f"  Improvement: +{best_val_acc - 78.33:.2f}%")

In [ ]:

print("=" * 80)
print("TESTING FINAL MODEL ON TEST DATASET")
print("=" * 80)

import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

# Definiraj nazive klasa
class_names = ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']

print(f"Model: {sgd_model.__class__.__name__}")
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print(f"Test dataset: {len(test_loader)} batches, {len(test_dataset)} images")

# TESTING ON TEST SET
sgd_model.eval()
test_predictions = []
test_labels = []
test_probabilities = []

print("\nStarting testing.")
with torch.no_grad():
    test_bar = tqdm(test_loader, desc='Testiranje')
    
    for images, labels in test_bar:
        images, labels = images.to(device), labels.to(device)
        
        outputs = sgd_model(images)
        probabilities = F.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
        
        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        test_probabilities.extend(probabilities.cpu().numpy())


test_accuracy = accuracy_score(test_labels, test_predictions) * 100
precision, recall, f1, support = precision_recall_fscore_support(
    test_labels, test_predictions, average=None, zero_division=0
)

print("\n" + "=" * 60)
print("RESULTS OF TESTING")
print("=" * 60)
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Number of test images: {len(test_labels)}")

# Per-class results
print(f"\nPer-class results:")
for i, class_name in enumerate(class_names):
    print(f"  {class_name}:")
    print(f"    Precision: {precision[i]:.3f}")
    print(f"    Recall: {recall[i]:.3f}")
    print(f"    F1-Score: {f1[i]:.3f}")
    print(f"    Support: {support[i]} slika")


avg_precision = np.mean(precision)
avg_recall = np.mean(recall)
avg_f1 = np.mean(f1)

print(f"\nAverage metrics:")
print(f"  Average Precision: {avg_precision:.3f}")
print(f"  Average Recall: {avg_recall:.3f}")
print(f"  Average F1-Score: {avg_f1:.3f}")

print("\n" + "=" * 60)
print("SUMMARY OF PERFORMANCE")
print("=" * 60)
print(f"Validation Accuracy: {best_val_acc:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Difference Val-Test: {best_val_acc - test_accuracy:.2f}%")


print(f"\nFINAL RESULTS:")
print(f"   Starting accuracy: 78.33%")
print(f"   Final test accuracy: {test_accuracy:.2f}%")
print(f"   Improvement: +{test_accuracy - 78.33:.2f}%")



In [ ]:
# CONFUSION MATRIX AND DETAILED VISUALIZATIONS
print("=" * 80)
print("CONFUSION MATRIX AND DETAILED VISUALIZATIONS")
print("=" * 80)

# Kreiraj confusion matrix
cm = confusion_matrix(test_labels, test_predictions)
cm_normalized = confusion_matrix(test_labels, test_predictions, normalize='true')

# Nastavi figure sa subplotovima
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Detaljne Analize Modela - Test Rezultati', fontsize=16, fontweight='bold')

# 1. Confusion Matrix (apsolutne vrijednosti)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[0,0])
axes[0,0].set_title('Confusion Matrix (Brojevi)')
axes[0,0].set_ylabel('Stvarne Klase')
axes[0,0].set_xlabel('Predvidene Klase')

# 2. Confusion Matrix (postoci)
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Oranges',
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[0,1])
axes[0,1].set_title('Confusion Matrix (Postoci)')
axes[0,1].set_ylabel('Stvarne Klase')
axes[0,1].set_xlabel('Predvidene Klase')

# 3. Per-class metrije
metrics_data = {
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1
}
x_pos = np.arange(len(class_names))
bar_width = 0.25

for i, (metric_name, values) in enumerate(metrics_data.items()):
    axes[1,0].bar(x_pos + i*bar_width, values, bar_width, 
                  label=metric_name, alpha=0.8)

axes[1,0].set_xlabel('Klase')
axes[1,0].set_ylabel('Score')
axes[1,0].set_title('Metrije po Klasama')
axes[1,0].set_xticks(x_pos + bar_width)
axes[1,0].set_xticklabels([name.split()[0] for name in class_names], rotation=45)
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# 4. Training progress
if 'val_accuracies' in locals() and len(val_accuracies) > 0:
    epochs = range(1, len(val_accuracies) + 1)
    axes[1,1].plot(epochs, val_accuracies, 'o-', color='red', linewidth=2, 
                   markersize=6, label=f'Validation (Max: {max(val_accuracies):.2f}%)')
    if 'train_accuracies' in locals():
        axes[1,1].plot(epochs, train_accuracies, 's-', color='blue', linewidth=2,
                       markersize=4, alpha=0.7, label=f'Training (Final: {train_accuracies[-1]:.2f}%)')
    
    axes[1,1].axhline(y=88, color='green', linestyle='--', alpha=0.7, label='Cilj (88%)')
    axes[1,1].axhline(y=90, color='orange', linestyle='--', alpha=0.7, label='Izvrsno (90%)')
    axes[1,1].set_xlabel('Epoha')
    axes[1,1].set_ylabel('Accuracy (%)')
    axes[1,1].set_title('Training Progress')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
else:
    # Ako nemamo training podatke, prikazi sazetak
    summary_text = f"""
    SAZETAK REZULTATA:
    
    Validation Accuracy: {best_val_acc:.2f}%
    Test Accuracy: {test_accuracy:.2f}%
    
    Poboljsanje: +{test_accuracy - 78.33:.2f}%
    
    Model Status: {'PRODUKCIJSKI READY' if test_accuracy >= 85 else 'POTREBNO PODESAVANJE'}
    """
    axes[1,1].text(0.1, 0.5, summary_text, transform=axes[1,1].transAxes,
                   fontsize=12, verticalalignment='center',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
    axes[1,1].set_title('Sazetak Performansi')
    axes[1,1].axis('off')

plt.tight_layout()
plt.show()

# Dodatne analize
print("\n" + "=" * 60)
print("DODATNE ANALIZE")
print("=" * 60)

# Analiza gresaka
print("Najcesce greske:")
for i in range(len(class_names)):
    for j in range(len(class_names)):
        if i != j and cm[i,j] > 0:
            error_rate = cm[i,j] / np.sum(cm[i,:]) * 100
            if error_rate > 5:  # Prikazi samo znacajne greske
                print(f"  {class_names[i]} → {class_names[j]}: {cm[i,j]} slika ({error_rate:.1f}%)")

# Najbolja i najslabija klasa
best_class_idx = np.argmax(f1)
worst_class_idx = np.argmin(f1)

print(f"\nNajbolja klasa: {class_names[best_class_idx]} (F1: {f1[best_class_idx]:.3f})")
print(f"Najslabija klasa: {class_names[worst_class_idx]} (F1: {f1[worst_class_idx]:.3f})")

# Model robusnos
confidence_scores = np.max(test_probabilities, axis=1)
avg_confidence = np.mean(confidence_scores)
low_confidence_count = np.sum(confidence_scores < 0.7)

print(f"\nModel pouzdanost:")
print(f"  Prosjecna pouzdanost: {avg_confidence:.3f}")
print(f"  Broj predikcija s pouzdanosti < 70%: {low_confidence_count}/{len(confidence_scores)} ({low_confidence_count/len(confidence_scores)*100:.1f}%)")



In [ ]:
print("=" * 80)
print("ANALYSIS OF PERFORMANCE BY CLASS")
print("=" * 80)

# Generiraj classification report
report = classification_report(test_labels, test_predictions, 
                             target_names=class_names, output_dict=True)

print("CLASSIFICATION REPORT:")
print(classification_report(test_labels, test_predictions, target_names=class_names))

# Detailed analysis by class
print("\n" + "=" * 60)
print("DETAILED ANALYSIS BY CLASS")
print("=" * 60)

for i, class_name in enumerate(class_names):
    print(f"\nCLASS: {class_name}")
    print("-" * 40)

    # Basic statistics
    tp = cm[i, i]  # True positives
    fp = np.sum(cm[:, i]) - tp  # False positives
    fn = np.sum(cm[i, :]) - tp  # False negatives
    tn = np.sum(cm) - tp - fp - fn  # True negatives
    
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = recall[i]  # Recall = Sensitivity

    print(f"  Basic Statistics:")
    print(f"    Number of test images: {support[i]}")
    print(f"    Correctly classified: {tp}")
    print(f"    Misclassified as this class: {fp}")
    print(f"    Missed: {fn}")

    print(f"  Metrics:")
    print(f"    Precision: {precision[i]:.3f} ({precision[i]*100:.1f}%)")
    print(f"    Recall (Sensitivity): {recall[i]:.3f} ({recall[i]*100:.1f}%)")
    print(f"    F1-Score: {f1[i]:.3f}")
    print(f"    Specificity: {specificity:.3f} ({specificity*100:.1f}%)")
    
   

    if precision[i] < 0.8 and recall[i] >= 0.8:
        print(f"    High false positives - model is diagnosing {class_name} too lately")
    elif precision[i] >= 0.8 and recall[i] < 0.8:
        print(f"    High false negatives - model is missing cases of {class_name}")
    elif precision[i] < 0.8 and recall[i] < 0.8:
        print(f"    More data or better features needed for {class_name}")


print("\n" + "=" * 60)
print("ANALYSIS OF MOST COMMON ERRORS")
print("=" * 60)

confusion_pairs = []
for i in range(len(class_names)):
    for j in range(len(class_names)):
        if i != j and cm[i,j] > 0:
            error_rate = cm[i,j] / np.sum(cm[i,:])
            confusion_pairs.append((error_rate, cm[i,j], i, j))

# Sortiraj po error rate
confusion_pairs.sort(reverse=True)

print("Top 5 most problematic pairs:")
for idx, (error_rate, count, i, j) in enumerate(confusion_pairs[:5]):
    print(f"{idx+1}. {class_names[i]} → {class_names[j]}")
    print(f"   Number of errors: {count}")
    print(f"   Error rate: {error_rate*100:.1f}%")

    # Medicinsko objasnjenje zasto se mogu zamijeniti
    if 'Mild' in class_names[i] and 'Very mild' in class_names[j]:
        print("   Reason: Close stages of dementia - hard to distinguish")
    elif 'Non Demented' in class_names[i] and 'Very mild' in class_names[j]:
        print("   Reason: Early stages may resemble normal conditions")
    elif 'Moderate' in class_names[i] and 'Mild' in class_names[j]:
        print("   Reason: Overlapping symptoms between stages")
    print()


print("=" * 60)
print("OVERALL MODEL SUMMARY")
print("=" * 60)

avg_metrics = {
    'Precision': np.mean(precision),
    'Recall': np.mean(recall), 
    'F1-Score': np.mean(f1),
    'Accuracy': test_accuracy/100
}

print("Average performance:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.3f} ({value*100:.1f}%)")

print(f"\nModel Status:")
if test_accuracy >= 90:
    print("   Model is at the cutting edge of current research!")
elif test_accuracy >= 85:
    print("   Model is ready for real-world use!")
elif test_accuracy >= 80:
    print("   Solid performance, potential for further optimization")
else:
    print("   Further optimization needed")

print(f"\nProgress:")
print(f"  Initial accuracy: 78.33%")
print(f"  Final accuracy: {test_accuracy:.2f}%")
print(f"  Improvement: +{test_accuracy-78.33:.2f} percentage points")

print(f"\nMedical Applicability:")
if avg_metrics['F1-Score'] >= 0.9:
    print("  EXCELLENT for aiding diagnosis")
elif avg_metrics['F1-Score'] >= 0.8:
    print("  VERY GOOD for screening")
elif avg_metrics['F1-Score'] >= 0.7:
    print("  USEFUL as a supportive tool")
else:
    print("  Further optimization needed for clinical use")

print("\nModel has been successfully developed and tested!")

In [ ]:
# SNIMANJE FINALNOG MODELA I KREIRANJE SAZETKA
print("=" * 80)
print("SNIMANJE MODELA I KREIRANJE FINALNOG SAZETKA")
print("=" * 80)

import json
from datetime import datetime


import os
model_dir = "saved_models"
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print(f"Kreiran direktorij: {model_dir}")


timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"alzheimers_resnet50_{timestamp}"


model_path = os.path.join(model_dir, f"{model_name}.pth")
torch.save({
    'model_state_dict': sgd_model.state_dict(),
    'model_class': 'AlzheimersResNet50',
    'num_classes': 4,
    'dropout_rate': 0.5,  
    'validation_accuracy': best_val_acc,
    'test_accuracy': test_accuracy,
    'training_epochs': fine_tuning_epochs,
    'optimizer': 'AdamW',
    'learning_rate': 5e-06,
    'class_names': class_names,
    'timestamp': timestamp
}, model_path)

print(f"Model snimljen u: {model_path}")


summary = {
    "projekt_info": {
        "naziv": "Alzheimer's Klasifikacija s Progressive Unfreezing",
        "datum": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "autor": "Lana",
        "opis": "ResNet50 model za klasifikaciju stadija demencije"
    },
    "model_arhitektura": {
        "tip_modela": "AlzheimersResNet50",
        "bazni_model": "ResNet50 (pretrained)",
        "broj_klasa": 4,
        "dropout_rate": 0.5,
        "ukupno_parametara": int(total_params),
        "trenabilni_parametri": int(trainable_params),
        "postotak_trenabilnih": round(trainable_params/total_params*100, 1)
    },
    "podaci": {
        "izvor": "Data folder",
        "klase": class_names,
        "train_velicina": len(train_dataset),
        "validation_velicina": len(val_dataset),
        "test_velicina": len(test_dataset),
        "batch_size": batch_size,
        "podjela": "70-20-10 (train-val-test)"
    },
    "trening_parametri": {
        "optimizer": "AdamW",
        "learning_rate_pocetni": 5e-06,
        "learning_rate_finalni": 4e-06,
        "weight_decay": 0.01,
        "broj_epoha": fine_tuning_epochs,
        "scheduler": "ReduceLROnPlateau",
        "strategija": "Progressive Unfreezing",
        "unfreezing_epoha": 6,
        "gradient_clipping": 1.0
    },
    "rezultati": {
        "validation_accuracy": round(best_val_acc, 2),
        "test_accuracy": round(test_accuracy, 2),
        "poboljsanje_od_pocetka": round(test_accuracy - 78.33, 2),
        "prosjecna_precision": round(float(np.mean(precision)), 3),
        "prosjecna_recall": round(float(np.mean(recall)), 3),
        "prosjecna_f1": round(float(np.mean(f1)), 3),
        "generalizacija_gap": round(best_val_acc - test_accuracy, 2)
    },
    "rezultati_po_klasama": {
        class_names[i]: {
            "precision": round(float(precision[i]), 3),
            "recall": round(float(recall[i]), 3),
            "f1_score": round(float(f1[i]), 3),
            "support": int(support[i])
        } for i in range(len(class_names))
    },
    "model_status": {
        "spremnost": "PRODUKCIJSKI READY" if test_accuracy >= 85 else "POTREBNO DODATNO PODESAVANJE",
        "medicinska_aplikabilnost": "IZVRSNO" if np.mean(f1) >= 0.9 else "VRLO DOBRO" if np.mean(f1) >= 0.8 else "DOBRO",
        "preporuke": [
            "Model je spreman za klinicku uporabu s nadzorem" if test_accuracy >= 90 else "Model je spreman za screening aplikacije",
            "Preporuca se dodatna validacija na vanjskim datasetima",
            "Potrebno je redovito pracenje performansi u produkciji"
        ]
    }
}


summary_path = os.path.join(model_dir, f"{model_name}_summary.json")
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"Sazetak snimljen u: {summary_path}")


print("\n" + "=" * 80)
print("FINALNI SAZETAK PROJEKTA")
print("=" * 80)

print(f"POSTIGNUTO: {test_accuracy:.2f}% test accuracy")


print(f"\nKLJUCNI POKAZATELJI:")
print(f"  • Poboljsanje: +{test_accuracy-78.33:.2f} postotnih bodova")
print(f"  • Generalizacija: {'Izvrsna' if abs(best_val_acc - test_accuracy) < 2 else 'Dobra' if abs(best_val_acc - test_accuracy) < 5 else 'Umjerena'}")
print(f"  • Prosjecni F1-Score: {np.mean(f1):.3f}")
print(f"  • Model robusnos: {'Visoka' if np.mean(confidence_scores) > 0.8 else 'Umjerena'}")

print(f"\nMEDICINSKA VAZNOST:")
print(f"  • Preciznost dijagnoze: {np.mean(precision)*100:.1f}%")
print(f"  • Osjetljivost (recall): {np.mean(recall)*100:.1f}%")
print(f"  • Balansiranost klasa: {'Da' if max(f1) - min(f1) < 0.2 else 'Ne'}")

print(f"\nPRODUKCIJSKA SPREMNOST:")
print(f"  • Status: {summary['model_status']['spremnost']}")
print(f"  • Preporuce za: {summary['model_status']['medicinska_aplikabilnost']} uporabu")

print(f"\nSNIMLJENI FAJLOVI:")
print(f"  • Model: {model_path}")
print(f"  • Sazetak: {summary_path}")

print(f"Model je spreman za implementaciju i moze se koristiti za")
print(f"automatsku klasifikaciju stadija demencije iz MRI slika!")
print("=" * 80)

In [ ]:

print("=" * 80)
print("TESTIRANJE FINALNOG MODELA NA TEST SKUPU PODATAKA")
print("=" * 80)

import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np

# Definiraj nazive klasa
class_names = ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']

print(f"Model: {sgd_model.__class__.__name__}")
print(f"Najbolja validation accuracy: {best_val_acc:.2f}%")
print(f"Test skup: {len(test_loader)} bathes, {len(test_dataset)} slika")

# TESTIRANJE NA TEST SKUPU
sgd_model.eval()
test_predictions = []
test_labels = []
test_probabilities = []

print("\nPokretanje testiranja...")
with torch.no_grad():
    test_bar = tqdm(test_loader, desc='Testiranje')
    
    for images, labels in test_bar:
        images, labels = images.to(device), labels.to(device)
        
        outputs = sgd_model(images)
        probabilities = F.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)
        
        test_predictions.extend(predicted.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())
        test_probabilities.extend(probabilities.cpu().numpy())


test_accuracy = accuracy_score(test_labels, test_predictions) * 100
precision, recall, f1, support = precision_recall_fscore_support(
    test_labels, test_predictions, average=None, zero_division=0
)

print("\n" + "=" * 60)
print("REZULTATI TESTIRANJA")
print("=" * 60)
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Broj testiranih slika: {len(test_labels)}")

# Per-class rezultati
print(f"\nRezultati po klasama:")
for i, class_name in enumerate(class_names):
    print(f"  {class_name}:")
    print(f"    Precision: {precision[i]:.3f}")
    print(f"    Recall: {recall[i]:.3f}")
    print(f"    F1-Score: {f1[i]:.3f}")
    print(f"    Support: {support[i]} slika")

# Prosječne metrije
avg_precision = np.mean(precision)
avg_recall = np.mean(recall)
avg_f1 = np.mean(f1)

print(f"\nProsječne metrije:")
print(f"  Prosječna Precision: {avg_precision:.3f}")
print(f"  Prosječna Recall: {avg_recall:.3f}")
print(f"  Prosječna F1-Score: {avg_f1:.3f}")

print("\n" + "=" * 60)
print("SAŽETAK PERFORMANSI")
print("=" * 60)
print(f"Validation Accuracy: {best_val_acc:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(f"Razlika Val-Test: {best_val_acc - test_accuracy:.2f}%")

if abs(best_val_acc - test_accuracy) < 2:
    print("  razlika < 2%")
elif abs(best_val_acc - test_accuracy) < 5:
    print(" razlika < 5%")
else:
    print("Mogući overfitting - razlika > 5%")

print(f"\n FINALNO:")
print(f"   Početna accuracy: 78.33%")
print(f"   Finalna test accuracy: {test_accuracy:.2f}%") 
print(f"   Poboljšanje: +{test_accuracy - 78.33:.2f}%")

if test_accuracy >= 90:
    print("Test accuracy >= 90%")
elif test_accuracy >= 85:
    print(" Test accuracy >= 85%")
elif test_accuracy >= 80:
    print("Test accuracy >= 80%")
else:
    print("potrebno dodatno podešavanje")

In [ ]:

print("=" * 80)
print("CONFUSION MATRIX I DETALJNE VIZUALIZACIJE")
print("=" * 80)


cm = confusion_matrix(test_labels, test_predictions)
cm_normalized = confusion_matrix(test_labels, test_predictions, normalize='true')


fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Detaljne Analize Modela - Test Rezultati', fontsize=16, fontweight='bold')

# 1. Confusion Matrix (apsolutne vrijednosti)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[0,0])
axes[0,0].set_title('Confusion Matrix (Brojevi)')
axes[0,0].set_ylabel('Stvarne Klase')
axes[0,0].set_xlabel('Predviđene Klase')

# 2. Confusion Matrix (postotci)
sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='Oranges',
            xticklabels=class_names, yticklabels=class_names,
            ax=axes[0,1])
axes[0,1].set_title('Confusion Matrix (Postotci)')
axes[0,1].set_ylabel('Stvarne Klase')
axes[0,1].set_xlabel('Predviđene Klase')


metrics_data = {
    'Precision': precision,
    'Recall': recall,
    'F1-Score': f1
}
x_pos = np.arange(len(class_names))
bar_width = 0.25

for i, (metric_name, values) in enumerate(metrics_data.items()):
    axes[1,0].bar(x_pos + i*bar_width, values, bar_width, 
                  label=metric_name, alpha=0.8)

axes[1,0].set_xlabel('Klase')
axes[1,0].set_ylabel('Score')
axes[1,0].set_title('Metrike po Klasama')
axes[1,0].set_xticks(x_pos + bar_width)
axes[1,0].set_xticklabels([name.split()[0] for name in class_names], rotation=45)
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)


if 'val_accuracies' in locals() and len(val_accuracies) > 0:
    epochs = range(1, len(val_accuracies) + 1)
    axes[1,1].plot(epochs, val_accuracies, 'o-', color='red', linewidth=2, 
                   markersize=6, label=f'Validation (Max: {max(val_accuracies):.2f}%)')
    if 'train_accuracies' in locals():
        axes[1,1].plot(epochs, train_accuracies, 's-', color='blue', linewidth=2,
                       markersize=4, alpha=0.7, label=f'Training (Final: {train_accuracies[-1]:.2f}%)')
    
    axes[1,1].axhline(y=88, color='green', linestyle='--', alpha=0.7, label='Cilj (88%)')
    axes[1,1].axhline(y=90, color='orange', linestyle='--', alpha=0.7, label='Izvrsno (90%)')
    axes[1,1].set_xlabel('Epoha')
    axes[1,1].set_ylabel('Accuracy (%)')
    axes[1,1].set_title('Training Progress')
    axes[1,1].legend()
    axes[1,1].grid(True, alpha=0.3)
else:

    summary_text = f"""
    SAŽETAK REZULTATA:
    
    Validation Accuracy: {best_val_acc:.2f}%
    Test Accuracy: {test_accuracy:.2f}%
    
    Poboljšanje: +{test_accuracy - 78.33:.2f}%
    
    Model Status: {'PRODUKCIJSKI READY' if test_accuracy >= 85 else 'POTREBNO PODEŠAVANJE'}
    """
    axes[1,1].text(0.1, 0.5, summary_text, transform=axes[1,1].transAxes,
                   fontsize=12, verticalalignment='center',
                   bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue"))
    axes[1,1].set_title('Sažetak Performansi')
    axes[1,1].axis('off')

plt.tight_layout()
plt.show()


print("\n" + "=" * 60)
print("DODATNE ANALIZE")
print("=" * 60)


print("Najčešće greške:")
for i in range(len(class_names)):
    for j in range(len(class_names)):
        if i != j and cm[i,j] > 0:
            error_rate = cm[i,j] / np.sum(cm[i,:]) * 100
            if error_rate > 5:  # Prikaži samo značajne greške
                print(f"  {class_names[i]} → {class_names[j]}: {cm[i,j]} slika ({error_rate:.1f}%)")


best_class_idx = np.argmax(f1)
worst_class_idx = np.argmin(f1)

print(f"\nNajbolja klasa: {class_names[best_class_idx]} (F1: {f1[best_class_idx]:.3f})")
print(f"Najslabija klasa: {class_names[worst_class_idx]} (F1: {f1[worst_class_idx]:.3f})")


confidence_scores = np.max(test_probabilities, axis=1)
avg_confidence = np.mean(confidence_scores)
low_confidence_count = np.sum(confidence_scores < 0.7)

print(f"\nModel pouzdanost:")
print(f"  Prosječna pouzdanost: {avg_confidence:.3f}")
print(f"  Broj predikcija s pouzdanosti < 70%: {low_confidence_count}/{len(confidence_scores)} ({low_confidence_count/len(confidence_scores)*100:.1f}%)")

print("\nAnaliza završena! Model je spreman za produkciju! ")

In [ ]:

print("=" * 80)
print("DETALJNE ANALIZE PERFORMANSI PO KLASAMA")
print("=" * 80)

# Generiraj classification report
report = classification_report(test_labels, test_predictions, 
                             target_names=class_names, output_dict=True)

print("CLASSIFICATION REPORT:")
print(classification_report(test_labels, test_predictions, target_names=class_names))

# Detaljne analize po klasama
print("\n" + "=" * 60)
print("DETALJNE ANALIZE PO KLASAMA")
print("=" * 60)

for i, class_name in enumerate(class_names):
    print(f"\nKLASA: {class_name}")
    print("-" * 40)
    
    # Osnovne statistike
    tp = cm[i, i]  # True positives
    fp = np.sum(cm[:, i]) - tp  # False positives
    fn = np.sum(cm[i, :]) - tp  # False negatives
    tn = np.sum(cm) - tp - fp - fn  # True negatives
    
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    sensitivity = recall[i]  # Recall = Sensitivity
 
    print(f"    Broj slika u testu: {support[i]}")
    print(f"    Točno klasificiranih: {tp}")
    print(f"    Pogrešno kao ova klasa: {fp}")
    print(f"    Propuštenih: {fn}")
  
    print(f"    Precision: {precision[i]:.3f} ({precision[i]*100:.1f}%)")
    print(f"    Recall (Sensitivity): {recall[i]:.3f} ({recall[i]*100:.1f}%)")
    print(f"    F1-Score: {f1[i]:.3f}")
    print(f"    Specificity: {specificity:.3f} ({specificity*100:.1f}%)")
    
    
    

    if precision[i] < 0.8 and recall[i] >= 0.8:
        print(f" Puno false positives - model prekasno dijagnostificira {class_name}")
    elif precision[i] >= 0.8 and recall[i] < 0.8:
        print(f" Puno false negatives - model propušta slučajeve {class_name}")
    elif precision[i] < 0.8 and recall[i] < 0.8:
        print(f" Potrebno više podataka ili bolje značajke za {class_name}")


print("\n" + "=" * 60)
print("ANALIZA NAJČEŠĆIH GREŠAKA")
print("=" * 60)

confusion_pairs = []
for i in range(len(class_names)):
    for j in range(len(class_names)):
        if i != j and cm[i,j] > 0:
            error_rate = cm[i,j] / np.sum(cm[i,:])
            confusion_pairs.append((error_rate, cm[i,j], i, j))


confusion_pairs.sort(reverse=True)

print("Top 5 najproblematičnijih parova:")
for idx, (error_rate, count, i, j) in enumerate(confusion_pairs[:5]):
    print(f"{idx+1}. {class_names[i]} → {class_names[j]}")
    print(f"   Broj grešaka: {count}")
    print(f"   Postotak grešaka: {error_rate*100:.1f}%")
    

    if 'Mild' in class_names[i] and 'Very mild' in class_names[j]:
        print(" Bliske faze demencije - teške za razlikovanje")
    elif 'Non Demented' in class_names[i] and 'Very mild' in class_names[j]:
        print(" Rane faze mogu biti slične normalnom stanju")
    elif 'Moderate' in class_names[i] and 'Mild' in class_names[j]:
        print(" Preklapanje simptoma između faza")
    print()


print("=" * 60)
print("SVEUKUPNI SAŽETAK MODELA")
print("=" * 60)

avg_metrics = {
    'Precision': np.mean(precision),
    'Recall': np.mean(recall), 
    'F1-Score': np.mean(f1),
    'Accuracy': test_accuracy/100
}

print("Prosječne performanse:")
for metric, value in avg_metrics.items():
    print(f"  {metric}: {value:.3f} ({value*100:.1f}%)")

print(f"\nModel Status:")
if test_accuracy >= 90:
    print(" Spreman za kliničku upotrebu s nadzorem")
elif test_accuracy >= 85:
    print(" Odlične performanse za medicinsku dijagnostiku")
elif test_accuracy >= 80:
    print(" Solidne performanse, moguca dodatna optimizacija")
else:
    print(" Potrebna dodatna optimizacija")

print(f"\nNapredak:")
print(f"  Početna accuracy: 78.33%")
print(f"  Finalna accuracy: {test_accuracy:.2f}%")
print(f"  Poboljšanje: +{test_accuracy-78.33:.2f} postotnih bodova")

print(f"\n Medicinska aplikabilnost:")
if avg_metrics['F1-Score'] >= 0.9:
    print(" IZVRSNO za pomoć u dijagnozi")
elif avg_metrics['F1-Score'] >= 0.8:
    print(" VRLO DOBRO za screening")
elif avg_metrics['F1-Score'] >= 0.7:
    print(" KORISNO kao pomoćni alat")
else:
    print(" Potrebno poboljšanje za kliničku upotrebu")

print("\nModel je uspješno razvijen i testiran!")

In [ ]:
# H5 EXPORT SA NAJBOLJIM MODELOM (93.75% ACCURACY)
import os
import h5py
import torch
import torch.nn as nn
import torchvision.models as models
from datetime import datetime
import numpy as np

print("H5 EXPORT SA NAJBOLJIM MODELOM")
print("=" * 50)

# 1. DEVICE SETUP
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# 2. MODEL CLASS DEFINITION
class AlzheimersResNet50(nn.Module):
    def __init__(self, num_classes=4, pretrained=True, dropout_rate=0.5):
        super(AlzheimersResNet50, self).__init__()
        
        # Koristi ResNet50 baseline
        self.resnet = models.resnet50(pretrained=pretrained)
        
        # Custom classifier sa regularizacijom
        num_features = self.resnet.fc.in_features
        
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.resnet(x)

# 3. UČITAJ NAJBOLJI MODEL SA 93.75% ACCURACY
best_model_path = r"saved_models\alzheimers_resnet50_20250809_165610.pth"

print(f"Učitavam najbolji model: {best_model_path}")

# Kreiraj model i učitaj težine
model_to_export = AlzheimersResNet50(num_classes=4, dropout_rate=0.5)

try:
    # Učitaj kompletan checkpoint
    checkpoint = torch.load(best_model_path, map_location=device)
    
    # Provjeri strukturu checkpoint-a
    if 'model_state_dict' in checkpoint:
        # Struktura s metadatima
        state_dict = checkpoint['model_state_dict']
        best_validation_acc = checkpoint.get('validation_accuracy', 93.65)
        best_test_acc = checkpoint.get('test_accuracy', 93.75)
        print(f"Učitan checkpoint sa metadatima")
    else:
        # Direktan state_dict
        state_dict = checkpoint
        best_validation_acc = 93.65
        best_test_acc = 93.75
        print(f"Učitan direktan state_dict")
    
    # Učitaj state_dict u model
    model_to_export.load_state_dict(state_dict)
    model_to_export = model_to_export.to(device)
    model_to_export.eval()
    
    print(f"USPJEŠNO UČITAN!")
    print(f"  Validation accuracy: {best_validation_acc}%")
    print(f"  Test accuracy: {best_test_acc}%")
    print(f"  Status: PRODUKCIJSKI READY")
    
except Exception as e:
    print(f"Greška učitavanja modela: {e}")
    print("Koristim osnovni model...")
    model_to_export = model_to_export.to(device)
    best_validation_acc = 0.0
    best_test_acc = 0.0

# 4. H5 EXPORT SETUP
export_dir = "exported_models"
if not os.path.exists(export_dir):
    os.makedirs(export_dir)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
h5_path = os.path.join(export_dir, f"alzheimers_BEST_model_{timestamp}.h5")

print(f"\nKreiranje NAJBOLJEG H5 modela: {os.path.basename(h5_path)}")

# 5. H5 DATOTEKA KREIRANJE
with h5py.File(h5_path, 'w') as h5f:
    # Metadata grupa sa PRAVOM accuracy
    meta_group = h5f.create_group('metadata')
    meta_group.attrs['model_name'] = 'AlzheimersResNet50_BEST'.encode('utf-8')
    meta_group.attrs['num_classes'] = 4
    meta_group.attrs['validation_accuracy'] = best_validation_acc
    meta_group.attrs['test_accuracy'] = best_test_acc  # PRAVA ACCURACY!
    meta_group.attrs['timestamp'] = timestamp.encode('utf-8')
    meta_group.attrs['pytorch_version'] = torch.__version__.encode('utf-8')
    meta_group.attrs['training_date'] = '2025-08-09'.encode('utf-8')
    meta_group.attrs['status'] = 'PRODUKCIJSKI_READY'.encode('utf-8')
    
    # Klase
    classes = ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']
    meta_group.create_dataset('classes', data=[s.encode('utf-8') for s in classes])
    
    # Model parametri grupa
    params_group = h5f.create_group('model_parameters')
    
    print("Spremam parametre NAJBOLJEG modela...")
    param_count = 0
    for name, param in model_to_export.state_dict().items():
        # Konvertiraj u numpy i spremi
        param_data = param.cpu().detach().numpy()
        
        # Provjeri da li je scalar (0D) - ne može biti kompresovan
        if param_data.ndim == 0:
            params_group.create_dataset(name, data=param_data)
        else:
            params_group.create_dataset(name, data=param_data, compression='gzip')
        param_count += 1
    
    print(f"Spremljeno {param_count} parametara")
    
    # Model arhitektura info
    arch_group = h5f.create_group('architecture')
    arch_group.attrs['base_model'] = 'ResNet50'.encode('utf-8')
    arch_group.attrs['pretrained'] = True
    arch_group.attrs['dropout_rate'] = 0.5
    arch_group.attrs['input_size'] = [224, 224, 3]
    arch_group.attrs['trainable_params'] = 17528580
    arch_group.attrs['total_params'] = 24690500
    
    # Preprocessing info
    preprocess_group = h5f.create_group('preprocessing')
    preprocess_group.attrs['input_size'] = [224, 224]
    preprocess_group.attrs['mean'] = [0.485, 0.456, 0.406]
    preprocess_group.attrs['std'] = [0.229, 0.224, 0.225]
    preprocess_group.attrs['normalize'] = True

print(f"\nSUCCESS - NAJBOLJI MODEL EXPORTIRAN!")
print(f"H5 model spremljen: {h5_path}")
print(f"Model validation accuracy: {best_validation_acc}%")
print(f"Model test accuracy: {best_test_acc}%")
print(f"Klase: {len(classes)}")
print(f"Veličina datoteke: {os.path.getsize(h5_path) / (1024*1024):.1f} MB")

# Provjera datoteke
print(f"\nPROVJERA H5 DATOTEKE:")
try:
    with h5py.File(h5_path, 'r') as verify_h5f:
        print(f"   Metadata: {len(verify_h5f['metadata'].attrs)} atributa")
        print(f"   Parametri: {len(verify_h5f['model_parameters'])} tensora")
        print(f"   Arhitektura: {len(verify_h5f['architecture'].attrs)} podataka")
        print(f"   Preprocessing: {len(verify_h5f['preprocessing'].attrs)} postavki")
        
        # Provjeri accuracy u metadati
        val_acc = verify_h5f['metadata'].attrs['validation_accuracy']
        test_acc = verify_h5f['metadata'].attrs['test_accuracy']
        print(f"   POTVRĐENO: Val accuracy: {val_acc}%, Test accuracy: {test_acc}%")
        print("   H5 datoteka je valjana sa PRAVOM accuracy!")
        
except Exception as e:
    print(f"Greška provjere: {e}")



In [ ]:
# TEST H5 MODELA NA SLIKAMA IZ DATA FOLDERA
import h5py
import numpy as np
from PIL import Image
import torch
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision.models as models
import os
import random
import matplotlib.pyplot as plt

print("TESTIRANJE H5 MODELA")
print("=" * 50)

# 1. UČITAJ H5 MODEL
h5_model_path = r"exported_models\alzheimers_BEST_model_20250814_161222.h5"

print(f"Učitavam H5 model: {h5_model_path}")

# Učitaj model metadata
with h5py.File(h5_model_path, 'r') as h5f:
    # Čitaj metadata
    val_acc = h5f['metadata'].attrs['validation_accuracy']
    test_acc = h5f['metadata'].attrs['test_accuracy']
    
    # Čitaj klase
    classes_bytes = h5f['metadata']['classes'][:]
    classes = [cls.decode('utf-8') for cls in classes_bytes]
    
    # Čitaj preprocessing informacije
    mean = h5f['preprocessing'].attrs['mean']
    std = h5f['preprocessing'].attrs['std']
    input_size = h5f['preprocessing'].attrs['input_size']
    
    print(f" Model učitan: Val acc {val_acc:.2f}%, Test acc {test_acc:.2f}%")
    print(f" Klase: {classes}")
    print(f" Input size: {input_size}")

# 2. REKONSTRUIRAJ PYTORCH MODEL
class AlzheimersResNet50(torch.nn.Module):
    def __init__(self, num_classes=4, pretrained=True, dropout_rate=0.5):
        super(AlzheimersResNet50, self).__init__()
        
        # Koristi ResNet50 baseline
        self.resnet = models.resnet50(pretrained=pretrained)
        
        # Custom classifier sa regularizacijom
        num_features = self.resnet.fc.in_features
        
        self.resnet.fc = torch.nn.Sequential(
            torch.nn.Dropout(dropout_rate),
            torch.nn.Linear(num_features, 512),
            torch.nn.BatchNorm1d(512),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(512, 256),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(256, num_classes)
        )
    
    def forward(self, x):
        return self.resnet(x)

# Kreiraj model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = AlzheimersResNet50(num_classes=4, dropout_rate=0.5).to(device)

# Učitaj parametre iz H5
print("Učitavam parametre iz H5...")
with h5py.File(h5_model_path, 'r') as h5f:
    state_dict = {}
    for param_name in h5f['model_parameters']:
        dataset = h5f['model_parameters'][param_name]
        

        if dataset.shape == ():  # Scalar
            param_data = dataset[()]  # Učitaj scalar vrijednost
        else:  # Array
            param_data = dataset[:]  # Učitaj cijeli array
            
        state_dict[param_name] = torch.from_numpy(np.array(param_data))

model.load_state_dict(state_dict)
model.eval()
print("✓ Model rekonstruiran i spreman za testiranje!")

# 3. PREPROCESSING TRANSFORMACIJE
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# 4. FUNKCIJA ZA PREDIKCIJU
def predict_image(image_path, model, transform, classes, device):
    """Predvidi klasu za jednu sliku"""
    try:
        # Učitaj sliku
        image = Image.open(image_path).convert('RGB')
        
        # Preprocess
        input_tensor = transform(image).unsqueeze(0).to(device)
        
        # Predikcija
        with torch.no_grad():
            outputs = model(input_tensor)
            probabilities = F.softmax(outputs, dim=1)
            predicted_idx = torch.argmax(outputs, dim=1).item()
            confidence = probabilities[0][predicted_idx].item()
        
        # Rezultati
        predicted_class = classes[predicted_idx]
        all_probs = probabilities[0].cpu().numpy()
        
        return predicted_class, confidence, all_probs, image
    
    except Exception as e:
        print(f"Greška kod {image_path}: {e}")
        return None, None, None, None


print("\n" + "=" * 60)
print("TESTIRANJE NA RANDOM SLIKAMA")
print("=" * 60)

data_path = "Data"
test_results = []

for class_name in classes:
    class_folder = os.path.join(data_path, class_name)
    if os.path.exists(class_folder):
        # Uzmi random sliku iz foldera
        images = [f for f in os.listdir(class_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if images:
            random_image = random.choice(images)
            image_path = os.path.join(class_folder, random_image)
            
            print(f"\nTestiram: {class_name}")
            print(f"Slika: {random_image}")
            
            predicted, confidence, probs, img = predict_image(
                image_path, model, test_transform, classes, device
            )
            
            if predicted:
                # Rezultat
                correct = predicted == class_name
                status = "✓ TOČNO" if correct else "✗ POGREŠNO"
                
                print(f"Stvarna klasa: {class_name}")
                print(f"Predviđena: {predicted} ({confidence:.2%}) {status}")
                
                # Top 3 predikcije
                top_indices = np.argsort(probs)[::-1][:3]
                print("Top 3 predikcije:")
                for i, idx in enumerate(top_indices):
                    print(f"  {i+1}. {classes[idx]}: {probs[idx]:.2%}")
                
                test_results.append({
                    'true_class': class_name,
                    'predicted': predicted,
                    'confidence': confidence,
                    'correct': correct,
                    'image_path': image_path
                })

# 6. SAŽETAK TESTIRANJA
print("\n" + "=" * 60)
print("SAŽETAK TESTIRANJA")
print("=" * 60)

if test_results:
    correct_predictions = sum(1 for r in test_results if r['correct'])
    total_tests = len(test_results)
    accuracy = correct_predictions / total_tests
    
    print(f"Testirane slike: {total_tests}")
    print(f"Točne predikcije: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2%}")
    print(f"Model validation accuracy: {val_acc:.2f}%")
    
    # Po klasama
    print(f"\nRezultati po klasama:")
    for class_name in classes:
        class_results = [r for r in test_results if r['true_class'] == class_name]
        if class_results:
            class_correct = sum(1 for r in class_results if r['correct'])
            avg_conf = np.mean([r['confidence'] for r in class_results])
            print(f"  {class_name}: {class_correct}/{len(class_results)} točno, avg confidence: {avg_conf:.2%}")



In [ ]:
# INSTALACIJA H5PY BIBLIOTEKE
print("=" * 60)
print("INSTALACIJA H5PY ZA HDF5 PODRŠKU")
print("=" * 60)

import subprocess
import sys

try:
    import h5py
    print("h5py je već instaliran!")
except ImportError:
    print("Instaliram h5py...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "h5py"])
        print("h5py uspješno instaliran!")
        
        # Provjeri instalaciju
        import h5py
        print(f"h5py verzija: {h5py.__version__}")
    except Exception as e:
        print(f"Greška pri instalaciji: {e}")
        print("Pokušavam alternativnu instalaciju...")
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "h5py", "--no-cache-dir"])
            import h5py
            print("h5py uspješno instaliran s alternativnom metodom!")
        except Exception as e2:
            print(f"Neuspješna instalacija: {e2}")
            print("Molim ručno instalirajte: pip install h5py")

print("=" * 60)

In [ ]:
print("BRZA KONFIGURACIJA ZA PROGRESSIVE UNFREEZING")
print("=" * 60)

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import numpy as np
from tqdm import tqdm
import os

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Data transforms (konzistentni sa prethodnim treningom)
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees=0, translate=(0.15, 0.15)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.1))
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Dataset setup
data_path = r"Data"
print(f"Loading data from: {data_path}")

# Kreiranje dataseta
full_dataset = datasets.ImageFolder(root=data_path, transform=train_transform)
print(f"Total images: {len(full_dataset)}")
print(f"Classes: {full_dataset.classes}")

# Dataset splitting (70-20-10 split)
total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

# Generator za konzistentno dijeljenje
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size], generator=generator
)

# Različite transformacije za val i test
val_dataset.dataset = datasets.ImageFolder(root=data_path, transform=val_transform)
test_dataset.dataset = datasets.ImageFolder(root=data_path, transform=val_transform)

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(val_dataset)}")  
print(f"Test size: {len(test_dataset)}")

# Data loaders (optimizirani batch size)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

print(f"Batch size: {batch_size}")
print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

# Model definition (AlzheimersResNet50 - kao iz prethodnog uspjesnog treninga)
class AlzheimersResNet50(nn.Module):
    def __init__(self, num_classes=4, pretrained=True, dropout_rate=0.7):
        super(AlzheimersResNet50, self).__init__()
        
        # Koristi ResNet50 baseline
        self.resnet = models.resnet50(pretrained=pretrained)
        
        # Zaleđi sve slojeve inicijalno (za maksimalnu regularizaciju)
        for param in self.resnet.parameters():
            param.requires_grad = False
        
        # Custom classifier sa agresivnom regularizacijom
        num_features = self.resnet.fc.in_features
        
        self.resnet.fc = nn.Sequential(
            nn.Dropout(dropout_rate),  # Agressiven dropout
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
        # Odmrzni samo finale slojeve inicijalno
        for param in self.resnet.fc.parameters():
            param.requires_grad = True
    
    def unfreeze_more_layers(self, percentage=0.3):
        """Progressive unfreezing method"""
        total_layers = len(list(self.resnet.named_parameters()))
        layers_to_unfreeze = int(total_layers * percentage)
        
        # Unutar unfreeze_more_layers method
        all_params = list(self.resnet.named_parameters())
        # Unfreeze odozdo naviše (od kraja prema početku)
        for i in range(min(layers_to_unfreeze, len(all_params))):
            name, param = all_params[-(i+1)]  # Kreni od kraja
            if 'fc' not in name:  # Ne diraj finalne slojeve (već su unfrozen)
                param.requires_grad = True
                
    def get_trainable_params(self):
        """Vrati broj parametara koji se treniraju"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)
    
    def forward(self, x):
        return self.resnet(x)

# Kreiraj model sa istim postavkama kao uspjeni trening
print("\nKreiram model...")
model = AlzheimersResNet50(num_classes=4, pretrained=True, dropout_rate=0.7)
model = model.to(device)

# Prikaži informacije o modelu
total_params = sum(p.numel() for p in model.parameters())
trainable_params = model.get_trainable_params()
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")
print(f"Trainable ratio: {trainable_params/total_params*100:.1f}%")

# Loss function sa label smoothing (kao u prethodnom uspjesnom treningu)
criterion = nn.CrossEntropyLoss(label_smoothing=0.2)
